In [1]:
import gym
import gym_power_trading
import pandas as pd
import numpy as np
from pathlib import Path

In [21]:
file_name = 'data\AEP_PSGC1_AMP_long.h5'
path = Path(file_name).resolve()
uri = str(path)

In [23]:
df = pd.read_hdf(uri)
df.head()

HE
MARKET_DAY NODE          TYPE    VALUE HourEnding       
2021-07-01 AEP.PSGC1.AMP Gennode LMP   1           25.88
                                       2           24.88
                                       3           23.45
                                       4           22.47
                                       5           21.73

In [24]:
env = gym.make('gym_power_trading/PowerTrading-v0', render_mode="human", df=df, window_size=2, frame_bound=[1, 2]) 

SyntaxError: invalid syntax (__init__.py, line 2)